In [ ]:
# Julia Code.
# This code uses the POProject package to simulate invasion dynamics of life cycles.
# See PO.jl for the model.
# This code is then used to generate the data for Figure 4 in the paper.

using Pkg
Pkg.activate("POProject")
using Plots
using LinearAlgebra
using DifferentialEquations
using LSODA
using Random
using StatsBase
using Distributions
using DataFrames
using CSV
using ArgParse
include("POProject/PO.jl")

sols = Dict()

N_max = 105
R0cutoff = 1e-8

invaders = [1,10,54,45,7,77,103,20,13,5,28,3]

function example_parameters(w)
    if w == 1
        parameters = (N_max = 105, gamma = 1.0, N_U = 4, N_L = 2, N_S = 20, r_o = 6.8, r_f = 1.0, mixing = 0.0, LCtype = "N+1")
    elseif w == 2
        parameters = (N_max = 105, gamma = 1.0, N_U = 18, N_L = 2, N_S = 20, r_o = 6.8, r_f = 1.0, mixing = 0.0, LCtype = "N+1")
    elseif w == 3
        parameters = (N_max = 105, gamma = 1.0, N_U = 120, N_L = 28, N_S = 20, r_o = 6.8, r_f = 1.0, mixing = 0.0, LCtype = "N+1")
    elseif w == 4
        parameters = (N_max = 105, gamma = 1.0, N_U = 120, N_L = 120, N_S = 20, r_o = 6.8, r_f = 1.0, mixing = 0.0, LCtype = "N+1")
    end
    return parameters
end

for w in 1:4
    println(w)

    parameters = example_parameters(w)
    x = zeros(N_max,N_max)
    x[1,1] = r(parameters)[1]/(parameters.gamma * (pA(parameters)[1] .^2 + pB(parameters)[1] .^2))

    lifeCyclesPresent = []

    for z in 1:length(invaders)
        println(z)
        println(lifeCyclesPresent)
        invader = invaders[z]
        x[invader, 1] = 0.01 * sum(x * (1:N_max))
        lifeCyclesPresent = sort([lifeCyclesPresent; invader])

        names = (:N_max, :gamma, :N_U, :N_L, :N_S, :r_o, :r_f, :mixing, :LCtype, :lifeCyclesPresent)
        values = (parameters.N_max, parameters.gamma, parameters.N_U, parameters.N_L, parameters.N_S, parameters.r_o, parameters.r_f, parameters.mixing, parameters.LCtype, lifeCyclesPresent)
        p = NamedTuple{names}(values)

        u0 = x[lifeCyclesPresent,1:maximum(lifeCyclesPresent)]

        prob = ODEProblem(dynamicalModel!, u0, (0,5e7), p)
        sol =  solve(prob, lsoda(), reltol = 1e-12, abstol = 1e-12)
        u = sol.u[length(sol.t)]

        d = DataFrame(t = sol.t)
        for i in 1:length(lifeCyclesPresent)
            LC = lifeCyclesPresent[i]
            d[!, "LC" * string(LC)] = [sum(sol.u[k][i,:][1:LC] .* (1:LC)) for k in 1:length(sol.t)]
        end
        CSV.write("Data/Figure 4/example_invasion_" *string(w)*"_"*string(z)*".csv",d)

        x = zeros(N_max,N_max)
        for i in 1:(length(lifeCyclesPresent))
            x[lifeCyclesPresent[i],1:(lifeCyclesPresent[i])] = u[i,1:(lifeCyclesPresent[i])]
        end

        # exclude life cycles
        excludedLifeCycles = filter(i->((logR0(x,i,parameters) < - R0cutoff) && (sum((x * (1:N_max))[i]) < 1e-6)), 1:N_max)
        for n in excludedLifeCycles
            for i in 1:n
                x[n,i] = 0.0
            end
        end
        x = tril(x) 
        lifeCyclesPresent = sort(setdiff(lifeCyclesPresent, excludedLifeCycles))
    end
end

  Activating 

1


project at `~/Library/CloudStorage/GoogleDrive-danielcpjorge98@gmail.com/My Drive/PhD/MC_paper_rep/POProject`


1
Any[]


┌ Warning: Using arrays or dicts to store parameters of different types can hurt performance.
│ Consider using tuples instead.
└ @ SciMLBase /Users/dj0794/.julia/packages/SciMLBase/Xs8nt/src/performance_warnings.jl:33


2
Any[1]
3
Any[1]
4
Any[1, 54]
5
Any[1, 54]
6
Any[1, 54]
7
Any[1, 77]
8
Any[1, 103]
9
Any[1, 103]
10
Any[1, 103]
11
Any[1, 103]
12
Any[1, 103]
2
1
Any[]
2
Any[1]
3
Any[1, 10]
4
Any[1, 10, 54]
5
Any[1, 10, 45]
6
Any[1, 7, 10, 45]
7
Any[1, 7, 10, 45]
8
Any[1, 7, 10, 45]
9
Any[1, 7, 10, 20]
10
Any[1, 7, 10, 13, 20]
11
Any[1, 5, 7, 10, 13, 20]
12
Any[1, 5, 7, 10, 13, 20]
3
1
Any[]
2
Any[1]
3
Any[1, 10]
4
Any[1, 10, 54]
5
Any[1, 10, 45]
6
Any[1, 7, 10, 45]
7
Any[1, 7, 10, 45]
8
Any[1, 7, 10, 45]
9
Any[20]
10
Any[20]
11
Any[20]
12
Any[28]
4
1
Any[]
2
Any[1]
3
Any[1, 10]
4
Any[54]
5
Any[54]
6
Any[54]
7
Any[77]
8
Any[77, 103]
9
Any[20, 77, 103]
10
Any[13, 20, 77, 103]
11
Any[5, 13, 20, 77, 103]
12
Any[5, 13, 20, 28, 77, 103]
